In [5]:
import os
import shutil
import rasterio
from glob import glob

In [6]:
data = os.getcwd() + "\\data"
months_list = glob(data+"\\*_data")
months_list

['C:\\Users\\Hello\\Documents\\remote-sensing\\APIs\\SentinelSat\\data\\Godavari2019_data',
 'C:\\Users\\Hello\\Documents\\remote-sensing\\APIs\\SentinelSat\\data\\Godavari2020_data',
 'C:\\Users\\Hello\\Documents\\remote-sensing\\APIs\\SentinelSat\\data\\Krishna2019_data',
 'C:\\Users\\Hello\\Documents\\remote-sensing\\APIs\\SentinelSat\\data\\Krishna2020_data']

In [7]:
if not os.path.exists(os.getcwd()+"\\stacked_images"):
    stack_folder = os.makedirs(os.getcwd()+"\\stacked_images")
for month in months_list:
    dst = month.split('\\')[-1]
    if not os.path.exists(os.getcwd()+"\\stacked_images\\"+dst):
        os.makedirs(os.getcwd()+"\\stacked_images\\"+dst)

In [8]:
#stacking
for month in months_list:
    print(month)
    data_list = glob(month+"\\*")
    num_scenes = len(data_list)
    i = 0
    for scene in data_list:
        i = i + 1
        path  =  scene + "\\GRANULE"
        data_folder = glob(path + "\\*")
        data_folder[0] =  data_folder[0] + "\\IMG_DATA\\R10m"
        tifFiles = []
        tifFiles.append(glob(data_folder[0] + "\\*B08_10m.tif")[0])
        tifFiles.append(glob(data_folder[0] + "\\*B04_10m.tif")[0])
        tifFiles.append(glob(data_folder[0] + "\\*B03_10m.tif")[0])
        tifFiles.append(glob(data_folder[0] + "\\*B02_10m.tif")[0])
        with rasterio.open(tifFiles[0]) as src0:
            meta = src0.meta
        meta.update(count = len(tifFiles))
        dest = tifFiles[0].split('\\')[-1].split('.')[0].split('_')[:2]
        dest_name = dest[0] + "_" + dest[1] + ".tif"
        dest = data_folder[0] +"\\"+ dest_name
        if not os.path.exists(dest):
            with rasterio.open(dest, 'w', **meta) as dst:
                for id, layer in enumerate(tifFiles, start=1):
                    with rasterio.open(layer) as src1:
                        dst.write_band(id, src1.read(1))
        print("(" + str(i)  + "/" + str(num_scenes) + ") Stacked " + dest_name + "...")
        stack_dest_name = scene.split("\\")[-1].split('.')[0] + ".tif"
        stack_dest = os.getcwd() + "\\stacked_images\\"+ month.split('\\')[-1] + "\\" + stack_dest_name
        if not os.path.exists(stack_dest):
            shutil.copy2(dest, stack_dest)
        print("(" + str(i)  + "/" + str(num_scenes) + ") Copied to stacked_data ...")

C:\Users\Hello\Documents\remote-sensing\APIs\SentinelSat\data\Godavari2019_data
(1/4) Stacked T44QND_20190215T045909.tif...
(1/4) Copied to stacked_data ...
(2/4) Stacked T44QPD_20190215T045909.tif...
(2/4) Copied to stacked_data ...
(3/4) Stacked T44QPE_20190215T045909.tif...
(3/4) Copied to stacked_data ...
(4/4) Stacked T44QPE_20190215T045909.tif...
(4/4) Copied to stacked_data ...
C:\Users\Hello\Documents\remote-sensing\APIs\SentinelSat\data\Godavari2020_data
(1/3) Stacked T44QND_20200215T045901.tif...
(1/3) Copied to stacked_data ...
(2/3) Stacked T44QPD_20200215T045901.tif...
(2/3) Copied to stacked_data ...
(3/3) Stacked T44QPE_20200215T045901.tif...
(3/3) Copied to stacked_data ...
C:\Users\Hello\Documents\remote-sensing\APIs\SentinelSat\data\Krishna2019_data
(1/8) Stacked T44PMC_20190210T045941.tif...
(1/8) Copied to stacked_data ...
(2/8) Stacked T44PNC_20190210T045941.tif...
(2/8) Copied to stacked_data ...
(3/8) Stacked T44QMD_20190210T045941.tif...
(3/8) Copied to stacked_

In [ ]:
#cleanup data folder

for month in months_list:
    print(month)
    data_list = glob(month+"\\*")
    for scene in data_list:
        shutil.rmtree(scene)
        print("Deleted scene " + scene.split('\\')[-1].split('.')[0] + "...")